In [7]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Streaming WordCount") \
        .master("local[3]") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 3) \
        .getOrCreate()

In [8]:
spark

In [9]:
# Creamos un flujo de escucha sobre netcat en localhost:9999
lines_df = spark.readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", "9999") \
        .load()

22/05/02 16:47:41 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [10]:
# leemos las lineas y las pasamos a palabras. Sobre ellas, realizamos una agrupación count
from pyspark.sql.functions import explode, split
words_df = lines_df.select(explode(split(lines_df.value, ' ')).alias('palabra'))
counts_df = words_df.groupBy("palabra").count()

In [11]:
word_count_query = counts_df.writeStream \
        .format("console") \
        .outputMode("append") \
        .option("checkpointLocation", "chk-point-dir") \
        .start()

22/05/02 16:47:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;
Aggregate [palabra#59], [palabra#59, count(1) AS count#63L]
+- Project [palabra#59]
   +- Generate explode(split(value#56,  , -1)), false, [palabra#59]
      +- StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.TextSocketSourceProvider@4d3d634c, socket, org.apache.spark.sql.execution.streaming.sources.TextSocketTable@6c3911cb, [host=localhost, port=9999], [value#56]


In [6]:
word_count_query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----+
|palabra|count|
+-------+-----+
+-------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---------+-----+
|  palabra|count|
+---------+-----+
|    Spark|    1|
|    Ochoa|    1|
|Streaming|    1|
|       en|    1|
|   Severo|    1|
|       el|    1|
+---------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----+
|palabra|count|
+-------+-----+
|     El|    1|
|  Ochoa|    2|
|  Elche|    1|
| Severo|    2|
|   está|    1|
|     en|    2|
+-------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 